In [ ]:
def count_unique_words(corpus):
    words_freq = dict()
    for sentence in tqdm(corpus):
        # print(sentence)
        for word in str(sentence).split():
            if word not in words_freq:
                words_freq[word] = 1
            else:
                words_freq[word] += 1
    return words_freq
    
words_freq = count_unique_words(df["queryText"])
len(words_freq)

In [ ]:
import pandas as pd
from tqdm.auto import tqdm
import random
from transformers import AutoTokenizer

class DatasetHandler:
    def __init__(self, file_address):
        tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
        self.df = pd.read_csv(file_address)
        self.df["similar"] = 1
        self.package_to_id, self.id_to_package = self.label_packages(self.df["packageName"], start_id=len(tokenizer.get_vocab()))
        
    def label_packages(self, packages: list, start_id=0):
        package_to_id = dict()
        id_to_package = dict()
        id_counter = start_id
        for package in tqdm(packages, desc="Labeling Packages"):
            if package not in package_to_id:
                package_to_id[package] = id_counter
                id_to_package[id_counter] = package
                id_counter += 1
        return package_to_id, id_to_package
    
    def save_negative_sampled_dataset(self, neg_to_pos_ratio:int=4, output_name="neg_sampled_data.pkl", seed=0):
        random.seed(seed)
        negative_rows = []
        random_choices = list(self.package_to_id.keys())
        for query_text, package_name in tqdm(zip(self.df["queryText"], self.df["packageName"]), total=len(self.df["queryText"])):
            for _ in range(neg_to_pos_ratio):
                random_package = random.choice(random_choices)
                while random_package == package_name:
                    random_package = random.choice(random_choices)
                negative_rows.append({'queryText': query_text, "packageName": random_package, "similar": 0})
        neg_df = pd.DataFrame(negative_rows)
        new_df = pd.concat([self.df, neg_df], ignore_index=True)
        new_df.to_pickle("neg_sampled_data.pkl")
        return new_df

dataset_handler = DatasetHandler("data/SA_history_clicks_of_98.csv")

In [ ]:
# new_df = dataset_handler.save_negative_sampled_dataset(neg_to_pos_ratio=5)
new_df